In [22]:
import random
import itertools

def debruijn(kmers):
    result = {}
    for pattern in kmers:
        result.setdefault(pattern[:-1], [])
        result[pattern[:-1]].append(pattern[1:])
    return result

def calculate_degree(graph):
    degree = {}
    for node in graph:
        degree.setdefault(node, [0,0]) #[indegree, outdegree]
        degree[node][1] += len(graph[node])
        for target_node in graph[node]:
            degree.setdefault(target_node, [0,0])
            degree[target_node][0] += 1
    return degree

def calculate_start_node(degree):
    return [key for key in degree if degree[key][0] < degree[key][1]][0]

def eulerian_path(graph):
    stack = []
    path = []
    degree = calculate_degree(graph)
    current_node = calculate_start_node(degree)
    while True:
        if current_node in graph and len(graph[current_node]) > 0:
            stack.append(current_node)
            target_node = graph[current_node][0]
            graph[current_node].remove(target_node)
            current_node = target_node
        else:
            path.insert(0, current_node)
            if len(stack) > 0:
                current_node = stack.pop()
            else:
                break
    return path

def construct_string(path):
    string = path[0][0]
    for p in path[1:]:
        #print(p)
        string += p[0][-1]
    for p in path[-k-d:]:
        string += p[1][-1]
    return string

def de_bruijn_from_paired_reads(patterns, k):
    graph = {}
    for text in patterns:
        prefix = (text[:k-1], text[k+1:-1])
        suffix = (text[1:k], text[k+2:])
        if prefix not in graph.keys():
            graph[prefix] = []
        graph[prefix].append(suffix)
    path = eulerian_path(graph)
    #print(path)
    string = construct_string(path)
    return string

if __name__ == "__main__":
    k = 4
    d = 2

    # paired_string ='GAGA|TTGA TCGT|GATG CGTG|ATGT TGGT|TGAG GTGA|TGTT GTGG|GTGA TGAG|GTTG GGTC|GAGA GTCG|AGAT' # Expected Output = GTGGTCGTGAGATGTTGA
    paired_string ='ACAC|CTCT ACAT|CTCA CACA|TCTC GACA|TCTC' # Expected Output = GACACATCTCTCA
    patterns = paired_string.split() # Converts to List, like ['ACAC|CTCT', 'ACAT|CTCA', 'CACA|TCTC', 'GACA|TCTC']

    with open("input_reconstruct.txt") as f:
        numbers = f.readline().split()
        k = int(numbers[0])
        d = int(numbers[1])
        patterns = f.readlines()
        #patterns = [p.strip() for p in patterns]
    patterns = patterns[0].split()

    print('K & D =', k, d)
    print('Patterns =', patterns)
    result = de_bruijn_from_paired_reads(patterns, k)
    print(result)


K & D = 4 2
Patterns = ['ACAC|CTCT', 'ACAT|CTCA', 'CACA|TCTC', 'GACA|TCTC']
GACACATTCTCA


In [24]:
def debruijn(kmers):
    result = {}
    for pattern in kmers:
        result.setdefault(pattern[:-1], [])
        result[pattern[:-1]].append(pattern[1:])
    return result

def calculate_degree(graph):
    degree = {}
    for node in graph:
        degree.setdefault(node, [0,0]) #[indegree, outdegree]
        degree[node][1] += len(graph[node])
        for target_node in graph[node]:
            degree.setdefault(target_node, [0,0])
            degree[target_node][0] += 1
    return degree

def genome_path(patterns):
    genome = patterns[0]
    for pattern in patterns[1:]:
    	genome += pattern[-1]
    return genome

def maximal_non_branching_paths(graph):
    paths = []
    visited = [] 
    degree = calculate_degree(graph)
    for current_node in graph:
        if degree[current_node][0] != 1 or degree[current_node][1] != 1:  
            visited.append(current_node)
            if degree[current_node][1] > 0:
                for target_node in graph[current_node]:
                    path = [current_node, target_node]
                    while degree[target_node][0] == 1 and degree[target_node][1] == 1:
                        visited.append(target_node)
                        target_node = graph[target_node][0]
                        path.append(target_node)
                    paths.append(path)
    for current_node in graph:
        if degree[current_node][0] == 1 and degree[current_node][1] == 1:
            if current_node not in visited:
                visited.append(current_node)
                target_node = graph[current_node][0]
                cycle = [current_node]
                while degree[target_node][0] == 1 and degree[target_node][1] == 1:
                    visited.append(target_node)
                    cycle.append(target_node)
                    if current_node == target_node:
                        break
                    target_node = graph[target_node][0]
                paths.append(cycle)
    return paths

def ContigsFromReads(patterns):
    contigs = []
    graph = debruijn(patterns)
    #print(graph)
    paths = maximal_non_branching_paths(graph)
    #print(paths)
    for path in paths:
        contigs.append(genome_path(path))
    return contigs

if __name__ == "__main__":
    patterns = ['ATG', 'ATG', 'TGT', 'TGG', 'CAT', 'GGA', 'GAT', 'AGA']
    # with open("rosalind_ba3k.txt") as f:
    #     f = f.read().splitlines()
    #     patterns = []
    #     for i in range(len(f)):
    #         patterns.append(f[i])
    #     #print(patterns)
    contigs = ContigsFromReads(patterns)
    out = open("out_ba3k.txt", "w")
    #out.writelines(contigs)
    #out.close()
    for contig in contigs:
        out.write(contig)
        out.write("\n")
        print(contig, end=" ")
    out.close()

ATG ATG TGT TGGA CAT GAT AGA 

In [38]:
kmer_string = 'TGAG GACT CTGA ACTG CTGA'
patterns = kmer_string.split()

# patterns = ['ATG', 'ATG', 'TGT', 'TGG', 'CAT', 'GGA', 'GAT', 'AGA']
contigs = ContigsFromReads(patterns)

print(' '.join(str(e) for e in contigs))

TGAG GACTG CTGA CTGA
